# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.47.0


## Dataset


### Overview
The dataset that will be used in this experiment is from Kaggle. It is a tablular dataset that contains features related to polished diamonds such as the cut, carat weight and colour. The aim of the data is to use these features to predict the price of the polished diamond.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

In [3]:
# choose a name for experiment
experiment_name = 'DiamondPriceAutoML'

experiment=Experiment(ws, experiment_name)

found = False
key = "diamond-data"
description_text = "Dataset to predict the price of round cut polished diamonds"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("Dataset not found, please register dataset in the workspace")


df = dataset.to_pandas_dataframe()
df.describe()

,carat,depth,table,price,x,y,z
count,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000
mean,0.797935,61.749322,57.457251,3932.734294,5.731158,5.734526,3.538730
std,0.473999,1.432626,2.234549,3989.338447,1.121730,1.142103,0.705679
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


Split the data into a training set and a test set that we can use to evaluate the model


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.2, random_state=223)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The AutoML task is set to regression as we want to predict a numerical value (diamond price) from a set of features. We will use the compute cluster that already exists in the workspace to train the model. The experiment is set to time out after 30 minutes so that we don;t consume an indefinite amount of resources. The primary metric that will be optimised is normalized_root_mean_squared_error as we want to minimise the average squared error. As the dataset is not very large, 5 cross-validations are used. Featurisation is enabled as no data cleaning was done after reading in the data and inputting it to the AutoML model. Early stopping is also enabled.


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
print('Found existing cluster, use it.')

Found existing cluster, use it.


In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.5,
    "enable_early_stopping": True,
    "compute_target" : compute_target,   
    "primary_metric": 'normalized_root_mean_squared_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=dataset,
                             label_column_name="price",
                             **automl_settings)

In [7]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

SDK version: 1.47.0
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
DiamondPriceAutoML,AutoML_bc959c0c-7ece-4336-a6d3-238239814104,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

A total of 27 models were trained using AutoML. Of these, the best performing model was a Voting Ensemble. This makes sense as it uses multiple models to make a decision. 

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run_automl, fitted_model = remote_run.get_output()
print('Best Run: ' + str(best_run_automl.id))
RunDetails(best_run_automl).show()
print(best_run_automl)
print(fitted_model)

Best Run: AutoML_bc959c0c-7ece-4336-a6d3-238239814104_30


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Run(Experiment: DiamondPriceAutoML,
Id: AutoML_bc959c0c-7ece-4336-a6d3-238239814104_30,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScale

KeyError: 'log_files'

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

Test the best model


In [10]:
y_test = x_test.pop("price")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


[3109.93544528  697.288093   3063.71486314 8959.01183329 3209.49670515
 2322.83417439 1348.38301849 9506.70500635 1721.21344485 1207.58149215]
Model MAPE:
0.062335059409250555

Model Accuracy:
0.9376649405907495


In [12]:
#TODO: Save the best model
automlModel = best_run_automl.register_model(
    model_name='DiamondPrice-AutoML-Model2', 
    model_path = "./",
    description='AutoML model to predict the price of a polished round diamond')

best_run_automl.download_files(output_directory='automl-output')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
from azureml.core import Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice


env = Environment.from_conda_specification(name='project_environment', file_path='./automl-output/outputs/conda_env_v_1_0_0.yml')

inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="score.py",
)

service_name = 'automl-diamond-model2'

autoMLModel = Model(ws, 'DiamondPrice-AutoML-Model2')
print(autoMLModel)

# deployment_config = LocalWebservice.deploy_configuration(port=8890)

service = Model.deploy(ws, service_name, [autoMLModel], inference_config)
service.wait_for_deployment(show_output=True)
print(service.port)


Model(workspace=Workspace.create(name='udacity-nanodegree', subscription_id='2cf207d8-3d70-4cc3-a99f-cce99d8c1c7c', resource_group='UdacityNano'), name=DiamondPrice-AutoML-Model2, id=DiamondPrice-AutoML-Model2:1, version=1, tags={}, properties={})
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-01 09:46:48+00:00 Creating Container Registry if not exists.
2022-11-01 09:46:48+00:00 Registering the environment.
2022-11-01 09:46:49+00:00 Use the existing image.
2022-11-01 09:46:49+00:00 Generating deployment configuration.
2022-11-01 09:46:50+00:00 Submitting deployment to compute.
2022-11-01 09:46:53+00:00 Checking the status of deployment automl-diamond-model2..
2022-11-01 09:49:36+00:00 Checking the status of inference endpoint automl-diamond-model2.
Failed


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 7863a87f-ad40-4c21-90f8-c1754b4cc44e
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: automl-diamond-model2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can intera

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 7863a87f-ad40-4c21-90f8-c1754b4cc44e
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: automl-diamond-model2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image f71bc50a7691442cb5230b3bea6cde50.azurecr.io/azureml/azureml_719123b736aa361f0ee848dfd47b40c8 locally. P

In [29]:
from azureml.core.model import Model
#experiment_name = 'DiamondPriceAutoML'

#experiment=Experiment(ws, experiment_name)
#experiment.start_logging

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    print(Model.get_model_path(model.name))
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

DiamondPrice-AutoML-Model2 version: 1


ModelNotFoundException: ModelNotFoundException:
	Message: Model DiamondPrice-AutoML-Model2 not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/nano-proj2/code/Users/raeesah_mangera/CapstoneProject. For more info, set logging level to DEBUG.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Model DiamondPrice-AutoML-Model2 not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/nano-proj2/code/Users/raeesah_mangera/CapstoneProject. For more info, set logging level to DEBUG."
    }
}

In [31]:
import logging
logging.basicConfig(level=logging.INFO)
print(os.getenv('AZUREML_MODEL_DIR'))
model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
#print(model_path)
#print(Model.get_model_path(model_name='DiamondPrice-AutoML-Model2'))

None


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
model_name = 'DiamondPrice-AutoML-Model'
from azureml.core.run import Run
experiment.start_logging
ws = Run.get_context().experiment.workspace
model_obj = Model(ws, model_name )
model_path = model_obj.download(exist_ok = True)
model = joblib.load(model_path)

In [15]:
print(service.get_logs())


ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 01 Nov 2022 08:24:58 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '00df22e2-9026-4e23-aae1-0bd2d63a8812', 'x-ms-client-session-id': '33b16853-fa95-468b-ba07-afcef16e5e04', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-eastus2-01', 'x-request-time': '0.185', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/automl-diamond-model-CsUb95F2LES1Iws76mzeUA\' under resource group \'UdacityNano\' was not found. For mo

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 01 Nov 2022 08:24:58 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '00df22e2-9026-4e23-aae1-0bd2d63a8812', 'x-ms-client-session-id': '33b16853-fa95-468b-ba07-afcef16e5e04', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-eastus2-01', 'x-request-time': '0.185', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/automl-diamond-model-CsUb95F2LES1Iws76mzeUA\' under resource group \'UdacityNano\' was not fo

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
raw_data = [
                [6,0.24,'Very Good','J','VVS2',62.8,57,3.94,3.96,2.48], 
                [24322,2,'Premium','D','SI2',59.3,62,8.12,8.06,4.8]
            ]

input_payload = json.dumps({
    'data': raw_data
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
